In [ ]:
from astropy.io import ascii
list_of_galaxies = ascii.read('interacting_gals.csv', delimiter=',')  
#list_of_galaxies = ascii.read('aaa.csv', delimiter=',')  

import io
import requests
from astropy.table import Table
import urllib.parse
import matplotlib.pyplot as plt
import math

#Most queries start with a Base URL that you append your query parameters to.
ned_base_url = 'http://vo.ned.ipac.caltech.edu'
ned_sed_query_path = '/services/querySED?REQUEST=queryData'

#Create a string that contains the search radius you want. Let's do 5 arcsec
search_radius = str(5.0 / 3600.0)

#For each galaxy in our sample,
i = 0
num_of_galaxies = len(list_of_galaxies)
num_of_columns  = 5
num_of_rows     = math.ceil(num_of_galaxies/num_of_columns)
plot_colors     = ('r', 'b', 'g', 'c', 'm', 'k', 'y')
num_of_colors   = len(plot_colors)
plot_scale      = 2
plot_horizontal = plot_scale * 5 * num_of_columns
plot_vertical   = plot_scale * 3 * num_of_rows

plt.figure(figsize=(plot_horizontal,plot_vertical))

for galaxies in list_of_galaxies:
    
    i+=1
    
    #Create a string that contains position information.
    position = str(galaxies['ra'])+','+str(galaxies['dec'])
    
    #Now string the URL together with the position and size parameters. Each paramter is 
    #separated by an ampersand.
    ned_objects_url = ned_base_url+ned_sed_query_path+'&POS='+position+'&SIZE='+search_radius
    
    #Send the URL as a request to NED, and capture the response.
    ned_objects_response = requests.get(ned_objects_url)
    
    #If the response is OK,
    if ned_objects_response.status_code == 200:
        
        #Put the table that is returned from NED into an astropy table.
        ned_objects_table = Table.read(io.BytesIO(ned_objects_response.content))
        n = 0
        
        #Extract the Object name and access URL in the response, removing special characters.
        #Retrieve the SED using the Access URL, put it into an Astropy table, and plot the SED
        for object in ned_objects_table:
            object_name = (object['ObjectName']).decode()
            object_url  = ned_base_url+urllib.parse.quote(object['ACREF'], safe = '/,&,?,=')
            ned_data_response = requests.get(object_url)
            if ned_data_response.status_code == 200:
                ned_data_table = Table.read(io.BytesIO(ned_data_response.content))
                x=ned_data_table['DataSpectralValue']
                y=ned_data_table['DataFluxValue']
                j = n % num_of_colors
                plt.subplot(num_of_rows,num_of_columns,i)
                plt.plot(x, y, plot_colors[j], label=object_name)
                n+=1
            plt.xlim(1e11,1e16)
            #plt.ylim(1e-5,1e2)
    plt.yscale('log')
    plt.xscale('log')
    plt.legend()
    plt.title('(ra,dec)='+'('+position+')', fontsize=14)
plt.show()
